<h1 align="center"><font size="7"><strong>MACHINE LEARNING</strong></font></h1>

-----------

#### Nama : Dyah Ayu Kusuma Wardhani 
#### Nim  : 4112321015

# __OBJECTIVE__
Terdapat data historis performa siswa dan hasil rekomendasi beasiswa mereka. Setiap sampel data terdiri beberapa atribut yaitu, 
- Name,
- OverallGrade,
- Obedient,
- ResearchScore,
- ProjectScore,
- Recommend,

Bangunlah model prediktif berdasarkan data yang ada sehingga Anda dapat memprediksi calon siswa apakah mereka akan direkomendasikan untuk mendapatkan beasiswa atau tidak berdasarkan dari atribut performa mereka.


# __A. DATA RETRIEVAL__

-------------
    
## 1. Mengimport library-library yang dibutuhkan

In [33]:
#Import Library
import pandas as pd

#Menonaktifkan pemberitahuan atau peringatan
pd.options.mode.chained_assignment = None  # default='warn'

## 2. Mengload Data
Memanggil Dataset berupa file csv menggunakan library pandas. Dan memuatnya dalam sebuah dataframe bernama "df"


In [34]:
df = pd.read_csv('student_records.csv')
df 

,Name,OverallGrade,Obedient,ResearchScore,ProjectScore,Recommend
0,Henry,A,Y,90,85,Yes
1,John,C,N,85,51,Yes
2,David,F,N,10,17,No
3,Holmes,B,Y,75,71,No
4,Marvin,E,N,20,30,No
5,Simon,A,Y,92,79,Yes
6,Robert,B,Y,60,59,No
7,Trent,C,Y,75,33,No


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Name           8 non-null      object
 1   OverallGrade   8 non-null      object
 2   Obedient       8 non-null      object
 3   ResearchScore  8 non-null      int64 
 4   ProjectScore   8 non-null      int64 
 5   Recommend      8 non-null      object
dtypes: int64(2), object(4)
memory usage: 516.0+ bytes


__Keterangan Dataset :__ 

Dataset ini berisi 6 kolom yang bertipe data integer dan object. Setiap atribut memeiliki penjelasan sendiri, yaitu sebagai berikut:
- `Name` : Nama siswa
- `OverallGrade` : Total nilai yang diperoleh
- `Obedient` : Kepatuhan siswa selama bersekolah, __Y__ artinya _Yes_ dan __N__ artinya _No_
- `ResearchScore` : Nilai yang diperoleh siswa dalam tugas penelitian
- `ProjectScore` : Nilai project yang diperoleh oleh siswa
- `Recommend`   : Apakah mereka mendapat rekomendasi beasiswa

# __B. DATA PREPARATION__
----------------

## 1. Feature Extraction and Engineering
_Feature Extraction_ adalah proses pengambilan variabel atau fitur dari data asli untuk mengurangi volume data, lalu mengkonversinya menjadi fitur yang dapat digunakan dalam proses analisis atau machine learning.

_Feature Engineering_ adalah merupakan proses pemilihan, transformasi, dan pembuatan fitur yang paling relevan dari data asli untuk meningkatkan kinerja algoritma pembelajaran mesin. Proses ini melibatkan tiga langkah utama: pembuatan fitur, transformasi, dan pemilihan fitur


Memilih atribut `OverallGrade`, `Obedient`, `ResearchScore`, `ProjectScore` untuk digunakan dalam pembuatan model. Atribut-atribut ini akan dijadikan fitur-fitur yang akan digunakan model untuk mempelajari pola data.

Menjadikan atribut `Recommed` menjadi atribut yang dijadikan target yang ingin diprediksi atau sering disebut variaabel dependen.



In [36]:
# Memilih atribut yang dibutuhkan untuk dijadikan fitur lalu membuatnya menjadi Dataframe
feature_names = ['OverallGrade', 'Obedient', 'ResearchScore', 'ProjectScore']
training_features = df[feature_names]

# Memberikan label untuk variabel yang ingin diprediksi
outcome_name = ['Recommend']
outcome_labels = df[outcome_name]

In [37]:
# Melihat isi feature yang dijadikan model
training_features

,OverallGrade,Obedient,ResearchScore,ProjectScore
0,A,Y,90,85
1,C,N,85,51
2,F,N,10,17
3,B,Y,75,71
4,E,N,20,30
5,A,Y,92,79
6,B,Y,60,59
7,C,Y,75,33


In [38]:
# Melihat feature label
outcome_labels

,Recommend
0,Yes
1,Yes
2,No
3,No
4,No
5,Yes
6,No
7,No


In [39]:
# Memisahkan fitur numerik dan kategorikal
numeric_feature_names = ['ResearchScore', 'ProjectScore']
categoricial_feature_names = ['OverallGrade', 'Obedient']

## 2. Numeric Feature Scaling
Feature scaling adalah teknik untuk membuat skala data setiap fitur menjadi sama. 

Pada praktek ini kami menggunakan Standard Scaler untuk melakukan transform feature dengan mengskalakan ulang data agar memiliki rata-rata 0 dan dan variance 1, membuat datanya tidak memiliki unit awal. Tujuan dilakukan Standard Scaler adalah untuk membantu memastikan perbedaan skalar antar fitur tidak mempengaruhi hasil model nanti.

In [40]:
#Import library StandardScaller dari modul sklearn.prepocessing
from sklearn.preprocessing import StandardScaler

#Melakukan scaling/standarisasi data menggunakan StandarScaler
ss = StandardScaler()

#Menentukan skala pada data latih fitur numeric
ss.fit(training_features[numeric_feature_names])

#Melakukan penskalaan pada fitur-fitur numerik
training_features[numeric_feature_names] = ss.transform(training_features[numeric_feature_names])

In [41]:
#Menampilkan dataset yang telah di scaler
training_features

,OverallGrade,Obedient,ResearchScore,ProjectScore
0,A,Y,0.899583,1.376650
1,C,N,0.730648,-0.091777
2,F,N,-1.803390,-1.560203
3,B,Y,0.392776,0.772004
4,E,N,-1.465519,-0.998746
5,A,Y,0.967158,1.117516
6,B,Y,-0.114032,0.253735
7,C,Y,0.392776,-0.869179


__Ket :__<br>
Dapat dilihat dari tabel diatas, setelah proses scaling pada kolom numeric (`ResearchScore`,`ProjectScore`) nilainya telah berubah sehingga memiliki rata-rata 0 dan dan variance 1

## 3. Engineering Categorical Features

Engineering Categorical Features adalah proses memanipulasi data atau mengubah fitur-fitur categorical dapat digunakan secara efektif oleh model _machine lerning_. Fitur - fitur categorical dikonversi menjadi bentuk yang dapat diinterpretasikan dan digunakan oleh model.

Pada praktek ini kita melakukan __One Hot Encoding__, agar variabel categorical lebih mudah dimengerti oleh model _machine learning_

In [42]:
#Melakukan one-hot encoding pada fitur
training_features = pd.get_dummies(training_features, columns=categoricial_feature_names)

#Menampilkan data yang telah di encode
training_features

,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_B,OverallGrade_C,OverallGrade_E,OverallGrade_F,Obedient_N,Obedient_Y
0,0.899583,1.376650,True,False,False,False,False,False,True
1,0.730648,-0.091777,False,False,True,False,False,True,False
2,-1.803390,-1.560203,False,False,False,False,True,True,False
3,0.392776,0.772004,False,True,False,False,False,False,True
4,-1.465519,-0.998746,False,False,False,True,False,True,False
5,0.967158,1.117516,True,False,False,False,False,False,True
6,-0.114032,0.253735,False,True,False,False,False,False,True
7,0.392776,-0.869179,False,False,True,False,False,False,True


In [43]:
#Membuat daftar fitur categorical baru setelah proses engineering
categorical_engineered_features = list(set(training_features.columns) - set(numeric_feature_names))
categorical_engineered_features

['OverallGrade_F',
 'OverallGrade_E',
 'OverallGrade_B',
 'Obedient_Y',
 'OverallGrade_A',
 'Obedient_N',
 'OverallGrade_C']

# __C. MODELING__ 
Modeling adalah proses membangun model matematis atau komputasional yang merepresentasikan hubungan antara variabel input (fitur) dan variabel output (target) dalam suatu dataset. Tujuan dari modeling untuk menemukan pola yang sebelumnya tidak terlihat atau membuat keputusan dari data yang ada.

In [44]:
# Mengimport library untuk pembuatan model
from sklearn.linear_model import LogisticRegression
import numpy as np

## 1. Model defining
Pada praktik ini model yang digunakan adalah model Supervised Learning sederhana, yaitu model klasifikasi meggunakan algoritma  __Linear Regression__.

In [45]:
#Pendefinisian Model
lr = LogisticRegression() 

## 2. Training Model 
Training model adalah proses untuk menjalankan sebuah algoritma _machine learning_ untuk dapat mengolah dataset di dalamnya yang sudah terbagi menjadi data training dan mengoptimalkan algoritma untuk menemukan pola maupun output tertentu.

Fitur __Recommend__ merupakan fitur dijadikan target yang ingin diprediksi

In [46]:
#Melatih model
model = lr.fit(training_features, np.array(outcome_labels['Recommend']))
model

LogisticRegression()

## 3. Model Evaluation
Model evaluation adalah proses untuk mengevaluasi kinerja model _machine learning_ yang telah dibangun. Tujuan utamanya adalah untuk memahami seberapa baik atau seberapa buruk model tersebut dapat melakukan prediksi atau generalisasi pada data baru dengan melihat nilai akurasi dari model.

In [47]:
# Mengevaluasi sederhana pada data pelatihan
pred_labels = model.predict(training_features)
actual_labels = np.array(outcome_labels['Recommend'])

In [48]:
#Import library yang dibutuhkan untuk evaluasi model
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

# Evaluasi performa model
print('Accuracy:', float(accuracy_score(actual_labels, pred_labels))*100, '%')
print('Classification Stats:')
print(classification_report(actual_labels, pred_labels))

Accuracy: 100.0 %
Classification Stats:
              precision    recall  f1-score   support

          No       1.00      1.00      1.00         5
         Yes       1.00      1.00      1.00         3

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



Penjelasan :
- Hasil precision, recall dan f1-score pada Y (0 dan 1) menunjukan nilai 1.00 atau 100%. 
- Diperoleh juga hasil akurasi menggunakan model Logistic Regression sebesar 100%.



## 4. Model Deployment 
Model Deployment adalah proses mengimplementasikan model _machine learning_  yang telah dibangun dan dievaluasi ke dalam lingkungan produksi sehingga dapat digunakan secara nyata untuk membuat prediksi atau mengambil keputusan berdasarkan data baru. 

In [49]:
# import library yang digunakan untuk mendeploy model
import joblib
import os

In [50]:
#Menyimpan model yang telah dibuat  ke dalam file dengan nama "Model" dan "Scaler"
if not os.path.exists('Model'):
    os.mkdir('Model')
if not os.path.exists('Scaler'):
    os.mkdir('Scaler') 
    
#Menyimpan model yang digunakan    
joblib.dump(model, r'Model/model.pickle')

#Menyimpan model dengan fitur yang telah di scaler
joblib.dump(ss, r'Scaler/scaler.pickle') 

['Scaler/scaler.pickle']

# __D. Prediction in Action__
Prediction in action dapat dikatakan juga sebagai Model Inference.<br>
Model inference adalah model yang sudah kita latih akan dicoba pada data selain data yang sudah tersedia. Data yang dimaksud bisa berupa data buatan, ataupun data yang ada pada internet.

Pada model inference ini dibuat dataset yang serupa dengan dataset sebelumnya dengan total 5 kolom yaitu 'Name', 'OverallGrade', 'Obedient', 'ResearchScore', 'ProjectScore'. Selanjutnya membuat prediksi menggunakan model yang sudah dijalankan sebelumnya yaitu Linier Regression mengklasifikasi siswa yang di recomendasikan untuk mendapat beasiswa (ya atau no) pada dataset. 

## 1. Mengload Model

In [51]:
#Memanggil model dan scaler yang sebelumnya telah kita simpan
model = joblib.load(r'Model/model.pickle')
scaler = joblib.load(r'Scaler/scaler.pickle')

## 2. Membuat Dataframe baru
Buat dataframe baru dengan nama new_data, untuk mencoba model yang telah dibuat sebelumnya

In [52]:
#Membuat Data Baru untuk menguji model
new_data = pd.DataFrame([{'Name': 'Nathan', 'OverallGrade': 'F', 'Obedient': 'N', 'ResearchScore': 30, 'ProjectScore': 20},
                  {'Name': 'Thomas', 'OverallGrade': 'A', 'Obedient': 'Y', 'ResearchScore': 78, 'ProjectScore': 80},
                   {'Name': 'Arnold', 'OverallGrade': 'C', 'Obedient': 'N', 'ResearchScore': 45, 'ProjectScore': 90}])
new_data = new_data[['Name', 'OverallGrade', 'Obedient', 'ResearchScore', 'ProjectScore']]
new_data

,Name,OverallGrade,Obedient,ResearchScore,ProjectScore
0,Nathan,F,N,30,20
1,Thomas,A,Y,78,80
2,Arnold,C,N,45,90


## 3. Data Preparation
Dataframe yang baru dibuat tadi dilakukan preparation agar siap dan sesuai dengan model yang akan dicoba

In [53]:
#Mengambil Fitur data yang ingin diprediksi, yaitu fitur `OverallGrade`, `Obedient`, `ResearchScore`, `ProjectScore `
prediction_features = new_data[feature_names]

#Melakukan Scalling pada data baru menggunakan scaler yang sudah disimpan sebelumnya
prediction_features[numeric_feature_names] = scaler.transform(prediction_features[numeric_feature_names])

#Melakukan encode data baru, agar  lebih mudah dimengerti oleh model
prediction_features = pd.get_dummies(prediction_features, columns=categoricial_feature_names)

#Menampilkan data baru yang telah dilakukan scaling dan encode
prediction_features

,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_C,OverallGrade_F,Obedient_N,Obedient_Y
0,-1.127647,-1.430636,False,False,True,True,False
1,0.494137,1.160705,True,False,False,False,True
2,-0.620839,1.592595,False,True,False,True,False


In [54]:
# Menambahkan kolom fitur categorical yang tidak ada ke dalam prediction_features agar memiliki dimensi kolom yang sama dengan model yang telah dibuat tadi
current_categorical_engineered_features = set(prediction_features.columns) - set(numeric_feature_names)
missing_features = set(categorical_engineered_features) - current_categorical_engineered_features
for feature in missing_features:
    
    #Menambahkan nilai 0 kedalam fitur categorical yang tidak ada pada data baru
    prediction_features[feature] = [0] * len(prediction_features) 

#Menampilkan databaru yang siap diuji dengan model
prediction_features

,ResearchScore,ProjectScore,OverallGrade_A,OverallGrade_C,OverallGrade_F,Obedient_N,Obedient_Y,OverallGrade_E,OverallGrade_B
0,-1.127647,-1.430636,False,False,True,True,False,0,0
1,0.494137,1.160705,True,False,False,False,True,0,0
2,-0.620839,1.592595,False,True,False,True,False,0,0


Sebelum dilakukan modeling paastikan bahwa data yang digunakan untuk prediksi memiliki struktur yang sama dengan data yang untuk melatih model nantinya. Jika fitur-fitur data baru berbeda dengan fitur yang telah dilatih, model tidak dapat memprediksi data baru yang kita buat.

In [55]:
prediction_features = prediction_features[training_features.columns]
prediction_features.columns = training_features.columns

missing_features = set(training_features.columns)-set(prediction_features.columns)
if missing_features:
    for feature in missing_features :
        prediction_features[feature] = 0

## 4. Melakukan Modeling
Melakukan prediksi menggunakan sebuah model yang telah kita save tadi __"model"__ pada data __prediction_features__

In [56]:
predictions = model.predict(prediction_features)

## 5. Menapilkan Hasil Prediksi
Menampilkan hasil prediksi dari data baru yang kita buat menggunakan model _Linier Regression_

In [57]:
#Menambahkan kolom 'Recommend' ke dalam DataFrame new_data dan mengisinya dengan nilai prediksi dalam variabel predictions.
new_data['Recommend'] = predictions

#Menampilkan Hasil Prediction
new_data

,Name,OverallGrade,Obedient,ResearchScore,ProjectScore,Recommend
0,Nathan,F,N,30,20,No
1,Thomas,A,Y,78,80,Yes
2,Arnold,C,N,45,90,Yes


**Kesimpulan** :
- Nathan dengan `OverallGrade` F, `Obedient` "No", `ResearchScore` 30 dan `ProjectScore` 20, __Tidak direkomendasikan__ untuk mendapat beasiswa
-  Thomas dengan `OverallGrade` A, `Obedient` "Yes", `ResearchScore` 78 dan `ProjectScore` 80, __Direkomendasikan__ untuk mendapat beasiswa
-  Arnold dengan `OverallGrade` C, `Obedient` "No", `ResearchScore` 45 dan `ProjectScore` 90, __Direkomendasikan__ untuk mendapat beasiswa